# ⚛️ Módulo 3: Mecánica Molecular y Campos de Fuerza
## Actividad 3.2: Campos de Fuerza en la Práctica

<div align="center">
  
**Universidad de Caldas - Departamento de Química**  
*Introducción a la Química Computacional (173G7G)*  
**Profesor:** José Mauricio Rodas Rodríguez

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maurorodas/Quimica_computacional_173G7G/blob/main/modulo_03_mecanica_molecular/02_campos_fuerza_practica.ipynb)

</div>

---

## 🎯 Objetivos de Aprendizaje

Al finalizar esta actividad, serás capaz de:
- Aplicar diferentes campos de fuerza a sistemas moleculares reales
- Comparar resultados obtenidos con MMFF94, UFF, GAFF y otros campos de fuerza
- Identificar cuándo usar cada tipo de campo de fuerza
- Evaluar la calidad de las geometrías optimizadas
- Calcular energías de conformación con diferentes campos de fuerza
- Comprender las limitaciones de cada campo de fuerza
- Seleccionar el campo de fuerza apropiado según el sistema y objetivo

---

## 📚 Introducción: Campos de Fuerza en la Práctica

> **Nota:** Si necesitas repasar los fundamentos teóricos de los campos de fuerza, las ecuaciones y componentes energéticos detallados, revisa la [Actividad 3.1: Fundamentos de Mecánica Molecular](01_fundamentos_mecanica_molecular.ipynb).

En esta actividad nos enfocamos en la **aplicación práctica** de diferentes campos de fuerza, comparando su rendimiento en sistemas reales y aprendiendo a seleccionar el más apropiado según el tipo de molécula y objetivo del cálculo.

### Principales Campos de Fuerza

| Campo de Fuerza | Año | Aplicación Principal | Ventajas | Limitaciones |
|-----------------|-----|---------------------|----------|--------------|
| **MMFF94** | 1996 | Moléculas orgánicas pequeñas | Balance precisión/velocidad | Limitado a orgánicos |
| **UFF** | 1992 | Tabla periódica completa | Universal, todos los elementos | Menos preciso |
| **AMBER** | 1984-presente | Proteínas, ácidos nucleicos | Excelente para biomoléculas | Requiere preparación |
| **CHARMM** | 1983-presente | Biomoléculas, membranas | Bien parametrizado | Complejo de configurar |
| **GAFF** | 2004 | Moléculas orgánicas generales | Compatible con AMBER | Requiere carga AM1-BCC |
| **OPLS-AA** | 1996 | Líquidos, proteínas | Bueno para líquidos | Pesado computacionalmente |
| **GROMOS** | 1996 | Biomoléculas en solución | Rápido para MD | Menos usado actualmente |

### Referencias Clave

1. **Halgren, T. A.** (1996). *Merck Molecular Force Field (MMFF94)*. J. Comp. Chem., 17, 490-519.

2. **Rappé, A. K. et al.** (1992). *UFF: A Full Periodic Table Force Field*. J. Am. Chem. Soc., 114, 10024-10035.

3. **Wang, J. et al.** (2004). *GAFF: General AMBER Force Field*. J. Comp. Chem., 25, 1157-1174.

4. **MacKerell, A. D. et al.** (1998). *All-Atom Empirical Potential for Molecular Modeling*. J. Phys. Chem. B, 102, 3586-3616.

## 🔧 Configuración del Entorno

In [ ]:
# Verificar si estamos en Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✓ Ejecutando en Google Colab")
    print("📦 Instalando paquetes necesarios...")
    
    # Instalar RDKit
    !pip install -q rdkit
    
    # Instalar py3Dmol para visualización
    !pip install -q py3Dmol
    
    # Instalar MMTK (opcional, para cálculos adicionales)
    !pip install -q MMTK
    
    print("✓ Paquetes instalados correctamente\n")
else:
    print("✓ Ejecutando localmente")
    print("⚠️  Asegúrate de tener instalados: rdkit, py3Dmol")

In [ ]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# RDKit para química computacional
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, Descriptors
from rdkit.Chem import rdMolTransforms, rdDistGeom
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PyMol

# Visualización 3D
import py3Dmol

# Configuración de matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Librerías importadas correctamente")
print(f"✓ Versión de RDKit: {Chem.__version__}")

## 🧪 Parte 1: Comparación de Campos de Fuerza - Moléculas Simples

### Caso de Estudio: n-Butano

Vamos a comparar diferentes campos de fuerza optimizando la geometría del n-butano y calculando su perfil conformacional.

In [ ]:
# Crear la molécula de n-butano
smiles = "CCCC"
mol_butano = Chem.MolFromSmiles(smiles)
mol_butano = Chem.AddHs(mol_butano)

# Generar geometría 3D inicial
AllChem.EmbedMolecule(mol_butano, randomSeed=42)

print("✓ Molécula de n-butano creada")
print(f"  Fórmula: {Chem.rdMolDescriptors.CalcMolFormula(mol_butano)}")
print(f"  Número de átomos: {mol_butano.GetNumAtoms()}")
print(f"  Número de conformaciones: {mol_butano.GetNumConformers()}")

# Visualizar estructura 2D
Draw.MolToImage(mol_butano, size=(300, 300))

In [ ]:
# Función para visualizar moléculas en 3D
def visualizar_3d(mol, width=400, height=400, style='stick'):
    """
    Visualiza una molécula en 3D usando py3Dmol
    
    Args:
        mol: Molécula RDKit
        width: Ancho del visor
        height: Alto del visor
        style: Estilo de visualización ('stick', 'sphere', 'line')
    """
    viewer = py3Dmol.view(width=width, height=height)
    
    # Convertir a bloque MOL
    mol_block = Chem.MolToMolBlock(mol)
    
    # Agregar molécula al visor
    viewer.addModel(mol_block, 'mol')
    
    # Establecer estilo
    viewer.setStyle({style: {}})
    
    # Configurar fondo y zoom
    viewer.setBackgroundColor('white')
    viewer.zoomTo()
    
    return viewer

# Visualizar n-butano inicial
print("Geometría inicial de n-butano:")
visualizar_3d(mol_butano)

### Optimización con Diferentes Campos de Fuerza

In [ ]:
# Diccionario para almacenar resultados
resultados = {}

# Lista de campos de fuerza disponibles en RDKit
campos_fuerza = ['MMFF94', 'MMFF94s', 'UFF']

# Copiar molécula original para cada campo de fuerza
moleculas = {}
for ff in campos_fuerza:
    moleculas[ff] = Chem.Mol(mol_butano)

print("🔬 Optimizando geometrías con diferentes campos de fuerza...\n")

for ff in campos_fuerza:
    mol = moleculas[ff]
    
    # Optimizar con el campo de fuerza correspondiente
    if ff == 'UFF':
        resultado = AllChem.UFFOptimizeMolecule(mol, maxIters=500)
        # Calcular energía
        ff_obj = AllChem.UFFGetMoleculeForceField(mol)
        energia = ff_obj.CalcEnergy()
    else:  # MMFF94 o MMFF94s
        resultado = AllChem.MMFFOptimizeMolecule(mol, mmffVariant=ff, maxIters=500)
        # Calcular energía
        props = AllChem.MMFFGetMoleculeProperties(mol, mmffVariant=ff)
        ff_obj = AllChem.MMFFGetMoleculeForceField(mol, props)
        energia = ff_obj.CalcEnergy()
    
    # Guardar resultados
    resultados[ff] = {
        'converged': resultado == 0,
        'energia': energia,
        'mol': mol
    }
    
    print(f"Campo de Fuerza: {ff}")
    print(f"  Convergencia: {'✓ Sí' if resultado == 0 else '✗ No'}")
    print(f"  Energía: {energia:.4f} kcal/mol")
    print()

# Crear tabla comparativa
df_resultados = pd.DataFrame({
    'Campo de Fuerza': campos_fuerza,
    'Energía (kcal/mol)': [resultados[ff]['energia'] for ff in campos_fuerza],
    'Convergió': [resultados[ff]['converged'] for ff in campos_fuerza]
})

print("📊 Tabla de Resultados:")
print(df_resultados.to_string(index=False))

### Análisis Geométrico: Comparación de Distancias de Enlace

In [ ]:
# Función para extraer distancias de enlace
def obtener_distancias_enlace(mol):
    """
    Extrae todas las distancias de enlace de una molécula
    """
    conf = mol.GetConformer()
    distancias = []
    enlaces = []
    
    for bond in mol.GetBonds():
        idx1 = bond.GetBeginAtomIdx()
        idx2 = bond.GetEndAtomIdx()
        
        atom1 = mol.GetAtomWithIdx(idx1)
        atom2 = mol.GetAtomWithIdx(idx2)
        
        dist = rdMolTransforms.GetBondLength(conf, idx1, idx2)
        
        enlace_tipo = f"{atom1.GetSymbol()}-{atom2.GetSymbol()}"
        
        distancias.append(dist)
        enlaces.append(enlace_tipo)
    
    return enlaces, distancias

# Analizar distancias para cada campo de fuerza
print("📏 Comparación de Distancias de Enlace (Å)\n")

datos_distancias = []

for ff in campos_fuerza:
    mol = resultados[ff]['mol']
    enlaces, distancias = obtener_distancias_enlace(mol)
    
    for enlace, dist in zip(enlaces, distancias):
        datos_distancias.append({
            'Campo de Fuerza': ff,
            'Tipo de Enlace': enlace,
            'Distancia (Å)': dist
        })

df_distancias = pd.DataFrame(datos_distancias)

# Calcular estadísticas por tipo de enlace
print("Promedios de distancias por tipo de enlace:\n")
resumen = df_distancias.groupby(['Tipo de Enlace', 'Campo de Fuerza'])['Distancia (Å)'].mean().unstack()
print(resumen)

In [ ]:
# Visualizar comparación de distancias C-C
fig, ax = plt.subplots(figsize=(10, 6))

# Filtrar solo enlaces C-C
df_cc = df_distancias[df_distancias['Tipo de Enlace'] == 'C-C']

# Crear gráfico de barras
sns.barplot(data=df_cc, x='Campo de Fuerza', y='Distancia (Å)', 
            palette='viridis', ax=ax)

# Añadir línea de referencia (valor experimental típico C-C)
ax.axhline(y=1.54, color='red', linestyle='--', linewidth=2, 
           label='Valor experimental (~1.54 Å)')

ax.set_title('Comparación de Distancias de Enlace C-C', fontsize=14, fontweight='bold')
ax.set_ylabel('Distancia (Å)', fontsize=12)
ax.set_xlabel('Campo de Fuerza', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🔄 Parte 2: Análisis Conformacional - Rotación del Enlace Central

Vamos a generar el perfil de energía de rotación alrededor del enlace central C2-C3 del n-butano.

In [ ]:
# Función para calcular energía en función del ángulo dihédrico
def calcular_perfil_torsional(mol, ff_type='MMFF94', angulo_inicial=0, angulo_final=360, paso=10):
    """
    Calcula el perfil de energía en función del ángulo dihédrico
    
    Args:
        mol: Molécula RDKit
        ff_type: Tipo de campo de fuerza ('MMFF94', 'MMFF94s', 'UFF')
        angulo_inicial: Ángulo inicial en grados
        angulo_final: Ángulo final en grados
        paso: Incremento de ángulo en grados
    
    Returns:
        angulos: Lista de ángulos
        energias: Lista de energías
    """
    # Identificar el ángulo dihédrico central (C1-C2-C3-C4)
    # Para butano: átomos 0-1-2-3 (carbonos)
    
    angulos = []
    energias = []
    
    for angulo in range(angulo_inicial, angulo_final, paso):
        # Crear una copia de la molécula
        mol_temp = Chem.Mol(mol)
        
        # Establecer el ángulo dihédrico
        conf = mol_temp.GetConformer()
        rdMolTransforms.SetDihedralDeg(conf, 0, 1, 2, 3, float(angulo))
        
        # Calcular energía
        if ff_type == 'UFF':
            ff_obj = AllChem.UFFGetMoleculeForceField(mol_temp)
        else:
            props = AllChem.MMFFGetMoleculeProperties(mol_temp, mmffVariant=ff_type)
            ff_obj = AllChem.MMFFGetMoleculeForceField(mol_temp, props)
        
        energia = ff_obj.CalcEnergy()
        
        angulos.append(angulo)
        energias.append(energia)
    
    return angulos, energias

print("🔄 Calculando perfiles conformacionales...")
print("⏳ Esto puede tomar un momento...\n")

# Calcular perfiles para cada campo de fuerza
perfiles = {}

for ff in campos_fuerza:
    mol = Chem.Mol(mol_butano)  # Usar molécula original
    angulos, energias = calcular_perfil_torsional(mol, ff_type=ff, paso=5)
    
    # Normalizar energías (restar el mínimo)
    energias = np.array(energias)
    energias = energias - energias.min()
    
    perfiles[ff] = {
        'angulos': angulos,
        'energias': energias
    }
    
    print(f"✓ Campo de fuerza {ff} completado")

print("\n✓ Cálculos completados")

In [ ]:
# Visualizar perfiles conformacionales
fig, ax = plt.subplots(figsize=(12, 7))

colores = {'MMFF94': '#2E86AB', 'MMFF94s': '#A23B72', 'UFF': '#F18F01'}
estilos = {'MMFF94': '-', 'MMFF94s': '--', 'UFF': '-.'}

for ff in campos_fuerza:
    ax.plot(perfiles[ff]['angulos'], 
            perfiles[ff]['energias'], 
            label=ff, 
            linewidth=2.5,
            color=colores[ff],
            linestyle=estilos[ff],
            marker='o',
            markersize=4,
            alpha=0.8)

# Marcar conformaciones importantes
ax.axvline(x=60, color='green', linestyle=':', alpha=0.5, label='Gauche')
ax.axvline(x=180, color='red', linestyle=':', alpha=0.5, label='Anti')
ax.axvline(x=300, color='green', linestyle=':', alpha=0.5)

ax.set_xlabel('Ángulo Dihédrico C1-C2-C3-C4 (°)', fontsize=12, fontweight='bold')
ax.set_ylabel('Energía Relativa (kcal/mol)', fontsize=12, fontweight='bold')
ax.set_title('Perfil Conformacional del n-Butano: Comparación de Campos de Fuerza', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 360)

# Añadir anotaciones
ax.text(0, ax.get_ylim()[1]*0.95, 'Eclipsada', ha='center', fontsize=10, style='italic')
ax.text(60, ax.get_ylim()[1]*0.95, 'Gauche', ha='center', fontsize=10, style='italic')
ax.text(180, ax.get_ylim()[1]*0.95, 'Anti', ha='center', fontsize=10, style='italic')

plt.tight_layout()
plt.show()

### Análisis de Barreras Energéticas

In [ ]:
# Calcular barreras energéticas y diferencias entre conformaciones
print("📊 Análisis de Barreras Energéticas\n")
print("="*70)

for ff in campos_fuerza:
    angulos = np.array(perfiles[ff]['angulos'])
    energias = np.array(perfiles[ff]['energias'])
    
    # Encontrar energía de conformación anti (alrededor de 180°)
    idx_anti = np.argmin(np.abs(angulos - 180))
    E_anti = energias[idx_anti]
    
    # Encontrar energía de conformación gauche (alrededor de 60°)
    idx_gauche = np.argmin(np.abs(angulos - 60))
    E_gauche = energias[idx_gauche]
    
    # Encontrar energía de conformación eclipsada (0°)
    idx_eclip = 0
    E_eclipsada = energias[idx_eclip]
    
    # Barrera de rotación (máxima energía)
    E_max = energias.max()
    
    print(f"\n{ff}:")
    print(f"  Conformación Anti (180°):        {E_anti:.3f} kcal/mol")
    print(f"  Conformación Gauche (60°):       {E_gauche:.3f} kcal/mol")
    print(f"  Conformación Eclipsada (0°):     {E_eclipsada:.3f} kcal/mol")
    print(f"  Barrera de rotación máxima:      {E_max:.3f} kcal/mol")
    print(f"  ΔE (Gauche - Anti):              {E_gauche - E_anti:.3f} kcal/mol")
    print(f"  ΔE (Eclipsada - Anti):           {E_eclipsada - E_anti:.3f} kcal/mol")

print("\n" + "="*70)
print("\n💡 Valores experimentales de referencia:")
print("   ΔE (Gauche - Anti):     ~0.6-0.9 kcal/mol")
print("   Barrera de rotación:    ~3-4 kcal/mol")

## 🧬 Parte 3: Sistemas Más Complejos - Comparación con Cicloalcanos

### Ciclohexano: Conformaciones Silla y Bote

In [ ]:
# Crear ciclohexano
smiles_ciclo = "C1CCCCC1"
mol_ciclo = Chem.MolFromSmiles(smiles_ciclo)
mol_ciclo = Chem.AddHs(mol_ciclo)

print("🔬 Ciclohexano")
print(f"  Fórmula: {Chem.rdMolDescriptors.CalcMolFormula(mol_ciclo)}")
print(f"  Número de átomos: {mol_ciclo.GetNumAtoms()}")

# Visualizar 2D
Draw.MolToImage(mol_ciclo, size=(300, 300))

In [ ]:
# Generar múltiples conformaciones
print("🔄 Generando conformaciones de ciclohexano...\n")

# Parámetros para generación de conformaciones
params = AllChem.ETKDGv3()
params.randomSeed = 42
params.numThreads = 0

# Generar múltiples conformaciones
num_conformaciones = 50
conf_ids = AllChem.EmbedMultipleConfs(mol_ciclo, numConfs=num_conformaciones, params=params)

print(f"✓ Se generaron {len(conf_ids)} conformaciones iniciales")

# Optimizar cada conformación con MMFF94
energias_mmff = []
for conf_id in conf_ids:
    resultado = AllChem.MMFFOptimizeMolecule(mol_ciclo, confId=conf_id, maxIters=500)
    
    # Calcular energía
    props = AllChem.MMFFGetMoleculeProperties(mol_ciclo)
    ff = AllChem.MMFFGetMoleculeForceField(mol_ciclo, props, confId=conf_id)
    energia = ff.CalcEnergy()
    energias_mmff.append(energia)

energias_mmff = np.array(energias_mmff)
energias_mmff_rel = energias_mmff - energias_mmff.min()

# Identificar conformaciones de baja energía
idx_ordenados = np.argsort(energias_mmff)
top_5 = idx_ordenados[:5]

print(f"\n📊 Top 5 conformaciones de menor energía (MMFF94):")
print("="*50)
for i, idx in enumerate(top_5, 1):
    print(f"  {i}. Conformación {idx}: {energias_mmff_rel[idx]:.3f} kcal/mol")

In [ ]:
# Visualizar conformación de menor energía
print("\n🔍 Visualización: Conformación de menor energía (Silla)")

conf_minima = top_5[0]
viewer = visualizar_3d(mol_ciclo, width=500, height=400, style='stick')

# Cambiar al conformador de menor energía
mol_temp = Chem.Mol(mol_ciclo)
mol_temp.RemoveAllConformers()
mol_temp.AddConformer(mol_ciclo.GetConformer(conf_minima))

visualizar_3d(mol_temp, width=500, height=400, style='stick')

In [ ]:
# Histograma de energías conformacionales
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(energias_mmff_rel, bins=20, color='skyblue', edgecolor='black', alpha=0.7)

ax.axvline(x=energias_mmff_rel.min(), color='green', linestyle='--', 
           linewidth=2, label='Conformación Silla')
ax.axvline(x=energias_mmff_rel[energias_mmff_rel > 5].min() if any(energias_mmff_rel > 5) else 6, 
           color='orange', linestyle='--', linewidth=2, label='Conformación Bote (~6 kcal/mol)')

ax.set_xlabel('Energía Relativa (kcal/mol)', fontsize=12, fontweight='bold')
ax.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
ax.set_title('Distribución de Energías Conformacionales del Ciclohexano', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Observaciones:")
print("  - La conformación silla es la más estable")
print("  - La conformación bote está ~5-6 kcal/mol por encima")
print("  - Valor experimental: ΔE(bote-silla) ≈ 5.5 kcal/mol")

## 💊 Parte 4: Aplicación a Fármacos - Ibuprofeno

### Comparación de Campos de Fuerza en Moléculas Farmacéuticas

In [ ]:
# Crear ibuprofeno
smiles_ibu = "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O"
mol_ibu = Chem.MolFromSmiles(smiles_ibu)
mol_ibu = Chem.AddHs(mol_ibu)

print("💊 Ibuprofeno")
print(f"  Fórmula: {Chem.rdMolDescriptors.CalcMolFormula(mol_ibu)}")
print(f"  Peso Molecular: {Descriptors.MolWt(mol_ibu):.2f} g/mol")
print(f"  Número de átomos: {mol_ibu.GetNumAtoms()}")
print(f"  Número de enlaces rotables: {Descriptors.NumRotatableBonds(mol_ibu)}")

# Visualizar 2D
Draw.MolToImage(mol_ibu, size=(400, 300))

In [ ]:
# Optimizar con diferentes campos de fuerza
print("🔬 Optimizando ibuprofeno con diferentes campos de fuerza...\n")

resultados_ibu = {}

for ff in campos_fuerza:
    # Crear copia y generar geometría 3D
    mol = Chem.Mol(mol_ibu)
    AllChem.EmbedMolecule(mol, randomSeed=42)
    
    # Optimizar
    if ff == 'UFF':
        resultado = AllChem.UFFOptimizeMolecule(mol, maxIters=1000)
        ff_obj = AllChem.UFFGetMoleculeForceField(mol)
        energia = ff_obj.CalcEnergy()
    else:
        resultado = AllChem.MMFFOptimizeMolecule(mol, mmffVariant=ff, maxIters=1000)
        props = AllChem.MMFFGetMoleculeProperties(mol, mmffVariant=ff)
        ff_obj = AllChem.MMFFGetMoleculeForceField(mol, props)
        energia = ff_obj.CalcEnergy()
    
    resultados_ibu[ff] = {
        'converged': resultado == 0,
        'energia': energia,
        'mol': mol
    }
    
    print(f"{ff}:")
    print(f"  Convergencia: {'✓' if resultado == 0 else '✗'}")
    print(f"  Energía: {energia:.4f} kcal/mol")
    print()

In [ ]:
# Analizar geometría del anillo aromático
print("📐 Análisis del anillo aromático\n")

for ff in campos_fuerza:
    mol = resultados_ibu[ff]['mol']
    
    # Obtener información del anillo
    rings = mol.GetRingInfo()
    ring_atoms = rings.AtomRings()[0]  # Primer anillo (aromático)
    
    # Calcular distancias de enlace en el anillo
    conf = mol.GetConformer()
    distancias_anillo = []
    
    for i in range(len(ring_atoms)):
        idx1 = ring_atoms[i]
        idx2 = ring_atoms[(i+1) % len(ring_atoms)]
        dist = rdMolTransforms.GetBondLength(conf, idx1, idx2)
        distancias_anillo.append(dist)
    
    print(f"{ff}:")
    print(f"  Distancia promedio C-C aromático: {np.mean(distancias_anillo):.4f} Å")
    print(f"  Desviación estándar: {np.std(distancias_anillo):.4f} Å")
    print()

print("💡 Valor experimental de enlace C-C aromático: ~1.39 Å")

## 🔬 Parte 5: Ejercicio Práctico - Análisis de tu Propia Molécula

### Ejercicio 1: Optimización de Cafeína

In [ ]:
# EJERCICIO: Optimiza la cafeína con diferentes campos de fuerza
# La cafeína tiene estructura compleja con anillos fusionados

# SMILES de cafeína
smiles_cafeina = "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"

# TU CÓDIGO AQUÍ:
# 1. Crea la molécula a partir del SMILES
# 2. Añade hidrógenos
# 3. Optimiza con MMFF94, MMFF94s y UFF
# 4. Compara las energías
# 5. Visualiza la estructura de menor energía

# Plantilla:
mol_cafeina = Chem.MolFromSmiles(smiles_cafeina)
mol_cafeina = Chem.AddHs(mol_cafeina)

print("☕ Cafeína")
print(f"  Fórmula: {Chem.rdMolDescriptors.CalcMolFormula(mol_cafeina)}")

# Continúa tu código aquí...

### Ejercicio 2: Perfil Conformacional del Etano

Calcula y grafica el perfil conformacional del etano (más simple que butano) rotando el enlace C-C.

In [ ]:
# EJERCICIO: Perfil conformacional del etano

smiles_etano = "CC"

# TU CÓDIGO AQUÍ:
# 1. Crea la molécula
# 2. Genera geometría 3D
# 3. Calcula el perfil torsional (ángulo H-C-C-H)
# 4. Grafica el resultado
# 5. Identifica las conformaciones eclipsada y escalonada

# Pistas:
# - Usa la función calcular_perfil_torsional (adaptar índices de átomos)
# - El etano tiene barrera de rotación ~3 kcal/mol
# - Eclipsada: 0°, 120°, 240°
# - Escalonada: 60°, 180°, 300°

### Ejercicio 3: Comparación de Ciclopropano vs Ciclobutano

Compara la tensión anular entre ciclopropano y ciclobutano.

In [ ]:
# EJERCICIO: Tensión anular

# Crear ciclopropano y ciclobutano
smiles_c3 = "C1CC1"
smiles_c4 = "C1CCC1"

# TU CÓDIGO AQUÍ:
# 1. Optimiza ambas moléculas con MMFF94
# 2. Calcula la energía de tensión por átomo de carbono
# 3. Compara con ciclopentano (C1CCCC1) y ciclohexano
# 4. Grafica energía vs tamaño de anillo

# Nota: Ciclopropano tiene mayor tensión por ángulos de 60° (vs 109.5° ideal)

## 📊 Parte 6: Comparación Exhaustiva de Campos de Fuerza

### Tabla Comparativa de Rendimiento

In [ ]:
# Crear tabla comparativa exhaustiva
import time

moleculas_test = {
    'Metano': 'C',
    'Etano': 'CC',
    'Propano': 'CCC',
    'Butano': 'CCCC',
    'Ciclopropano': 'C1CC1',
    'Ciclobutano': 'C1CCC1',
    'Benceno': 'c1ccccc1',
    'Tolueno': 'Cc1ccccc1'
}

print("🔬 Comparación exhaustiva de campos de fuerza\n")
print("="*80)

resultados_comparacion = []

for nombre, smiles in moleculas_test.items():
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    
    for ff in campos_fuerza:
        # Generar geometría
        mol_temp = Chem.Mol(mol)
        AllChem.EmbedMolecule(mol_temp, randomSeed=42)
        
        # Medir tiempo de optimización
        inicio = time.time()
        
        if ff == 'UFF':
            resultado = AllChem.UFFOptimizeMolecule(mol_temp, maxIters=500)
            ff_obj = AllChem.UFFGetMoleculeForceField(mol_temp)
            energia = ff_obj.CalcEnergy()
        else:
            resultado = AllChem.MMFFOptimizeMolecule(mol_temp, mmffVariant=ff, maxIters=500)
            props = AllChem.MMFFGetMoleculeProperties(mol_temp, mmffVariant=ff)
            ff_obj = AllChem.MMFFGetMoleculeForceField(mol_temp, props)
            energia = ff_obj.CalcEnergy()
        
        tiempo = time.time() - inicio
        
        resultados_comparacion.append({
            'Molécula': nombre,
            'Campo de Fuerza': ff,
            'Energía (kcal/mol)': energia,
            'Tiempo (s)': tiempo,
            'Convergió': resultado == 0,
            'Átomos': mol.GetNumAtoms()
        })

df_comparacion = pd.DataFrame(resultados_comparacion)

print("\n📊 Resultados completos:")
print(df_comparacion.to_string(index=False))

In [ ]:
# Visualización: Tiempo de cálculo vs tamaño del sistema
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Tiempo vs número de átomos
for ff in campos_fuerza:
    df_ff = df_comparacion[df_comparacion['Campo de Fuerza'] == ff]
    axes[0].scatter(df_ff['Átomos'], df_ff['Tiempo (s)'], 
                    label=ff, s=100, alpha=0.7)

axes[0].set_xlabel('Número de Átomos', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Tiempo de Optimización (s)', fontsize=12, fontweight='bold')
axes[0].set_title('Rendimiento Computacional', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Energías por molécula
df_pivot = df_comparacion.pivot(index='Molécula', columns='Campo de Fuerza', values='Energía (kcal/mol)')
df_pivot.plot(kind='bar', ax=axes[1], width=0.8)

axes[1].set_xlabel('Molécula', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Energía (kcal/mol)', fontsize=12, fontweight='bold')
axes[1].set_title('Comparación de Energías', fontsize=14, fontweight='bold')
axes[1].legend(title='Campo de Fuerza')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 🎓 Parte 7: Guía de Selección de Campos de Fuerza

### ¿Cuándo usar cada campo de fuerza?

In [ ]:
# Crear diagrama de flujo para selección de campo de fuerza
from matplotlib.patches import Rectangle, FancyBboxPatch, FancyArrowPatch
from matplotlib.patches import ConnectionPatch

fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')

# Colores
color_inicio = '#4ECDC4'
color_decision = '#FFE66D'
color_resultado = '#95E1D3'
color_warning = '#FF6B6B'

# Función auxiliar para crear cajas
def crear_caja(ax, x, y, ancho, alto, texto, color, style='round'):
    box = FancyBboxPatch((x-ancho/2, y-alto/2), ancho, alto,
                          boxstyle=f"{style},pad=0.1", 
                          facecolor=color, edgecolor='black', 
                          linewidth=2, alpha=0.8)
    ax.add_patch(box)
    ax.text(x, y, texto, ha='center', va='center', 
            fontsize=10, fontweight='bold', wrap=True)

# Título
ax.text(5, 9.5, 'Guía de Selección de Campo de Fuerza', 
        ha='center', fontsize=16, fontweight='bold')

# Diagrama de flujo
crear_caja(ax, 5, 8.5, 2.5, 0.6, '¿Qué tipo de\nsistema?', color_inicio)

# Rama 1: Moléculas orgánicas pequeñas
crear_caja(ax, 2, 7, 2, 0.6, 'Moléculas orgánicas\npequeñas (<100 átomos)', color_decision)
crear_caja(ax, 2, 5.5, 2, 0.6, 'Usa MMFF94\no MMFF94s', color_resultado)
ax.annotate('', xy=(2, 5.8), xytext=(2, 6.7),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

# Rama 2: Biomoléculas
crear_caja(ax, 5, 7, 2, 0.6, 'Proteínas/\nÁcidos nucleicos', color_decision)
crear_caja(ax, 5, 5.5, 2, 0.6, 'Usa AMBER,\nCHARMM u OPLS', color_resultado)
ax.annotate('', xy=(5, 5.8), xytext=(5, 6.7),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

# Rama 3: Materiales/Metales
crear_caja(ax, 8, 7, 2, 0.6, 'Materiales/\nComplejos metálicos', color_decision)
crear_caja(ax, 8, 5.5, 2, 0.6, 'Usa UFF o\nReaxFF', color_resultado)
ax.annotate('', xy=(8, 5.8), xytext=(8, 6.7),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

# Conectar inicio con decisiones
ax.annotate('', xy=(2, 7.3), xytext=(4.2, 8.2),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))
ax.annotate('', xy=(5, 7.3), xytext=(5, 8.2),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))
ax.annotate('', xy=(8, 7.3), xytext=(5.8, 8.2),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

# Consideraciones adicionales
crear_caja(ax, 2, 4, 1.8, 0.5, 'Ventajas:\n- Rápido\n- Preciso', color_resultado, 'round')
crear_caja(ax, 5, 4, 1.8, 0.5, 'Ventajas:\n- Parametrizado\n- Validado', color_resultado, 'round')
crear_caja(ax, 8, 4, 1.8, 0.5, 'Ventajas:\n- Universal\n- Flexible', color_resultado, 'round')

# Advertencia general
crear_caja(ax, 5, 1.5, 4, 0.8, 
           '⚠️ Ningún campo de fuerza describe reacciones químicas\nUsa métodos QM para ruptura/formación de enlaces', 
           color_warning, 'round')

plt.tight_layout()
plt.show()

## 📋 Resumen y Recomendaciones

### Tabla de Recomendaciones por Sistema

In [ ]:
# Crear tabla de recomendaciones
data_recomendaciones = {
    'Sistema': [
        'Moléculas orgánicas pequeñas',
        'Fármacos',
        'Hidrocarburos',
        'Proteínas',
        'Ácidos nucleicos (ADN/ARN)',
        'Lípidos y membranas',
        'Complejos organometálicos',
        'Cristales orgánicos',
        'Polímeros',
        'Sistemas con metales de transición'
    ],
    'Campo de Fuerza Recomendado': [
        'MMFF94/MMFF94s',
        'MMFF94 o GAFF',
        'MMFF94 u OPLS',
        'AMBER (ff14SB, ff19SB)',
        'AMBER (OL15, bsc1)',
        'CHARMM36 o Slipids',
        'UFF',
        'COMPASS o MMFF94',
        'OPLS-AA o COMPASS',
        'UFF'
    ],
    'Alternativa': [
        'UFF',
        'OPLS-AA',
        'CHARMM',
        'CHARMM36m',
        'CHARMM27',
        'AMBER lipid14',
        'Parámetros específicos',
        'UFF',
        'GAFF',
        'Parámetros específicos'
    ],
    'Observaciones': [
        'Balance óptimo precisión/velocidad',
        'GAFF si se usa con AMBER',
        'Excelente para alcanos/alquenos',
        'Altamente recomendado para MD',
        'Existen versiones mejoradas',
        'Simular membranas celulares',
        'Cobertura universal, menos preciso',
        'Propiedades mecánicas',
        'Propiedades de bulk',
        'Verificar parámetros disponibles'
    ]
}

df_recomendaciones = pd.DataFrame(data_recomendaciones)

print("📋 TABLA DE RECOMENDACIONES DE CAMPOS DE FUERZA")
print("="*100)
print(df_recomendaciones.to_string(index=False))
print("="*100)

### Comparación de Características Técnicas

In [ ]:
# Crear tabla comparativa de características
data_caracteristicas = {
    'Campo de Fuerza': ['MMFF94', 'MMFF94s', 'UFF', 'AMBER', 'CHARMM', 'OPLS-AA', 'GAFF'],
    'Año': [1996, 1996, 1992, 1984, 1983, 1996, 2004],
    'Tipos de Átomo': [94, 94, 118, '~30', '~50', '~200', '~30'],
    'Cargas': ['MMFF94', 'MMFF94s', 'QEq', 'RESP/AM1-BCC', 'TIP3P', 'OPLS', 'AM1-BCC'],
    'Parámetros vdW': ['Buffered 14-7', 'Buffered 14-7', 'Lennard-Jones', 'Lennard-Jones', 
                       'Lennard-Jones', 'Lennard-Jones', 'Lennard-Jones'],
    'Precisión Geom.': ['Alta', 'Alta', 'Media', 'Alta', 'Alta', 'Alta', 'Alta'],
    'Velocidad': ['Rápida', 'Rápida', 'Muy rápida', 'Media', 'Media', 'Media', 'Rápida'],
    'Software': ['RDKit, Schrödinger', 'RDKit, Schrödinger', 'Universal', 
                 'AmberTools, GROMACS', 'CHARMM, GROMACS', 'GROMACS, LAMMPS', 
                 'AmberTools']
}

df_caracteristicas = pd.DataFrame(data_caracteristicas)

print("\n🔍 CARACTERÍSTICAS TÉCNICAS DETALLADAS")
print("="*120)
print(df_caracteristicas.to_string(index=False))
print("="*120)

## 💡 Conclusiones y Mejores Prácticas

### Lecciones Aprendidas

### 📌 Conclusiones Principales

1. **No existe un campo de fuerza "mejor"** - cada uno tiene fortalezas y debilidades específicas

2. **MMFF94/MMFF94s**: 
   - Excelente para moléculas orgánicas pequeñas
   - Balance óptimo precisión/velocidad
   - Primera elección para drug design

3. **UFF**:
   - Universal pero menos preciso
   - Útil para screening inicial
   - Bueno para elementos inusuales

4. **Campos especializados (AMBER, CHARMM)**:
   - Necesarios para biomoléculas
   - Requieren preparación cuidadosa
   - Validados extensivamente

5. **Siempre valida tus resultados**:
   - Compara con datos experimentales cuando sea posible
   - Usa múltiples campos de fuerza si hay dudas
   - Considera métodos QM para validación

### ⚠️ Limitaciones Importantes

- **NO** modelan ruptura/formación de enlaces
- **NO** describen propiedades electrónicas
- **Dependientes** de la calidad de parametrización
- **Requieren validación** con datos experimentales o QM

### ✅ Mejores Prácticas

1. **Selección del campo de fuerza**:
   - Considera el tipo de sistema
   - Revisa la literatura
   - Verifica cobertura de tipos de átomo

2. **Optimización**:
   - Usa geometría inicial razonable
   - Verifica convergencia
   - Compara con múltiples campos si es crítico

3. **Validación**:
   - Compara geometrías con cristalografía
   - Verifica energías con datos experimentales
   - Usa cálculos QM de referencia cuando sea posible

4. **Documentación**:
   - Registra siempre el campo de fuerza usado
   - Anota versión del software
   - Reporta parámetros de optimización

## 🔗 Referencias y Recursos Adicionales

### Artículos Fundamentales

1. **Halgren, T. A.** (1996). *Merck Molecular Force Field (MMFF94)*. 
   - J. Comp. Chem., 17(5-6), 490-519.
   - DOI: 10.1002/(SICI)1096-987X(199604)17:5/6<490::AID-JCC1>3.0.CO;2-P

2. **Rappé, A. K., Casewit, C. J., et al.** (1992). *UFF: A Full Periodic Table Force Field*.
   - J. Am. Chem. Soc., 114(25), 10024-10035.
   - DOI: 10.1021/ja00051a040

3. **Wang, J., Wolf, R. M., et al.** (2004). *Development of a General AMBER Force Field*.
   - J. Comp. Chem., 25(9), 1157-1174.
   - DOI: 10.1002/jcc.20035

4. **MacKerell, A. D., et al.** (1998). *All-Atom Empirical Potential for Molecular Modeling*.
   - J. Phys. Chem. B, 102(18), 3586-3616.
   - DOI: 10.1021/jp973084f

5. **Jorgensen, W. L., et al.** (1996). *Development and Testing of the OPLS All-Atom Force Field*.
   - J. Am. Chem. Soc., 118(45), 11225-11236.
   - DOI: 10.1021/ja9621760

### Libros Recomendados

1. **Leach, A. R.** (2001). *Molecular Modelling: Principles and Applications*, 2nd ed.
   - Pearson Education. ISBN: 978-0582382107

2. **Jensen, F.** (2017). *Introduction to Computational Chemistry*, 3rd ed.
   - Wiley. ISBN: 978-1118825990

3. **Cramer, C. J.** (2004). *Essentials of Computational Chemistry*, 2nd ed.
   - Wiley. ISBN: 978-0470091821

### Recursos Online

- **RDKit Documentation**: https://www.rdkit.org/docs/
- **AMBER Force Fields**: http://ambermd.org/AmberModels.php
- **CHARMM Force Fields**: https://www.charmm.org/
- **CompChem Resources**: https://www.compchems.com/

### Software Relacionado

- **RDKit**: Open-source cheminformatics
- **Open Babel**: Conversión de formatos
- **Avogadro**: Editor molecular visual
- **PyMOL**: Visualización avanzada
- **AmberTools**: Suite completa AMBER (gratis)
- **GROMACS**: Simulaciones de dinámica molecular

## 📝 Cuestionario de Autoevaluación

Responde las siguientes preguntas para verificar tu comprensión:

1. **¿Qué es un campo de fuerza molecular?**

2. **¿Cuál es la principal diferencia entre MMFF94 y UFF?**

3. **¿Por qué los campos de fuerza no pueden describir reacciones químicas?**

4. **¿Cuándo usarías MMFF94 en lugar de AMBER?**

5. **¿Qué significa que un campo de fuerza sea "parametrizado"?**

6. **¿Por qué es importante validar los resultados de mecánica molecular?**

7. **¿Qué tipo de campo de fuerza usarías para simular una proteína en agua?**

8. **¿Cuál es la ventaja principal de UFF sobre otros campos de fuerza?**

9. **¿Qué información necesitas para optimizar una geometría molecular?**

10. **¿Cómo determinarías si una geometría optimizada es correcta?**

### Respuestas Esperadas (Revisa después de intentar responder)

<details>
<summary>Haz clic para ver las respuestas</summary>

1. Un conjunto de funciones matemáticas y parámetros que describen la energía de un sistema molecular

2. MMFF94 es específico para orgánicos y más preciso; UFF cubre toda la tabla periódica pero es menos preciso

3. Porque son métodos clásicos que no tratan electrones explícitamente

4. Para moléculas orgánicas pequeñas; AMBER es para biomoléculas grandes

5. Que sus constantes fueron ajustadas usando datos experimentales o QM de alta calidad

6. Para asegurar que los resultados son físicamente razonables y confiables

7. AMBER, CHARMM u OPLS-AA (campos especializados en biomoléculas)

8. Cubre todos los elementos de la tabla periódica

9. Estructura inicial, campo de fuerza apropiado, criterios de convergencia

10. Comparando con datos cristalográficos, experimentales, o cálculos QM de referencia

</details>

---

<div align="center">

## 🎉 ¡Felicitaciones!

Has completado la **Actividad 3.2: Campos de Fuerza en la Práctica**

Ahora comprendes:
- ✅ Cómo aplicar diferentes campos de fuerza
- ✅ Comparar resultados entre MMFF94, UFF y otros
- ✅ Seleccionar el campo apropiado para cada sistema
- ✅ Validar e interpretar geometrías optimizadas
- ✅ Las limitaciones de cada método

**Siguiente actividad**: Optimización de Geometrías Moleculares

[![Previous](https://img.shields.io/badge/⬅️_Actividad_3.1-Fundamentos_MM-blue.svg)](01_fundamentos_mecanica_molecular.ipynb)
[![Next](https://img.shields.io/badge/Actividad_3.3_➡️-Optimización_Geometrías-green.svg)](03_optimizacion_geometrias.ipynb)

---

📚 **[Volver al Módulo 3](README.md)** | 🏠 **[Inicio del Curso](../README.md)**

---

**Universidad de Caldas - Departamento de Química**  
*Química Computacional 173G7G - 2026*

</div>